In [1]:
# a script for making pretty plots

In [2]:
# imports
import matplotlib.pyplot as plt
import numpy as np
import csv as csv

# general 
import os
import sys
import glob
from functools import reduce
import re

from sklearn import linear_model, datasets

from numpy import polyval

import pandas as pd

%matplotlib inline

In [3]:
# function for loading one data-set
def load_data(file_path, skiprows_=1):
    data = np.loadtxt(file_path, delimiter = '\t', skiprows=skiprows_)
    # print(data.shape) 
    # handles some weird cases, e.g. when there is no data in the file
    if (len(data.shape) < 2):
        data = data[None, :]
    if (data.shape[1] == 0):
        I = np.array([0])
        z = np.array([0])
    else:
        I = data[:, -1]
        z = data[:, -2]
    return (z, I)

In [4]:
# returns the y coordinate for the given line
# y = kx + b; also works with vectors
def f_x(x_, k_, b_):
    return k_*x_ + b_

In [5]:
def g_x(a, x):
    return polyval(a, x)

In [13]:
# general setup for pretty images 
num_files = 16
lines_params = np.zeros([num_files, 2])
channel_id = 1 # 1,2,3
# folder_path = '/Users/kkolyva/Desktop/2018-04-18-08-29-25-test/test/2018-04-24-15-33-21-median-median-first-test/csv/' 
image = 'N2_dpy-23_ex_int_ama-1_016'
Ia_min, Ia_max = 0, 40000
za_min, za_max = 0, 55

# folder_path = '/Volumes/1TB/2018-05-15-12-30-27-SEA12-full-stack/'
folder_path = '/Volumes/MILKYKLIM⁩/⁨2019-07-08-exp⁩/N2⁩'
# folder_path = '/Volumes/1TB/2018-06-14-12-36-00-N2-full-stack/'

In [14]:
# possible labels 
stain = ['DPY-23_EX', 'WDR-5.2', 'MDH-1']
stage = 'E' # only embryos
comment = '' # only empty ones

In [15]:
# important indices
stain_columns = ['C0_stain', 'C1_stain', 'C2_stain', 'C3_stain', 'C4_stain']
type_columns = ['C0_type', 'C1_type', 'C2_type', 'C3_type', 'C4_type']
stain_prefix = np.array([['C1-', 'C2-', 'C3-', 'C4-', 'C5-']])
ext = '.csv'
filename_column = 'new filename'

In [16]:
# read the db and parse images that we want to process
df = pd.read_csv(folder_path + "smFISH-database/N2-Table 1.csv", sep=',', na_values=['']);
df.head()

FileNotFoundError: [Errno 2] File b'/Volumes/MILKYKLIM\xe2\x81\xa9/\xe2\x81\xa82019-07-08-exp\xe2\x81\xa9/N2\xe2\x81\xa9smFISH-database/N2-Table 1.csv' does not exist: b'/Volumes/MILKYKLIM\xe2\x81\xa9/\xe2\x81\xa82019-07-08-exp\xe2\x81\xa9/N2\xe2\x81\xa9smFISH-database/N2-Table 1.csv'

In [ ]:
# this is general
# filter to have only *good* and *embryo* files
good_indices = np.logical_and((df['stage'] == stage).tolist() , (df['comment'].isnull()).tolist())
good_indices.shape[0]
print(np.sum(good_indices == True))

In [ ]:
# choose necessary stains
dataset1 = []
df_good = (df[type_columns].astype(np.object) == stain[0]).loc[good_indices, :]
row, col = np.where(df_good)
n_samples = df.shape[0]
new_prefix = np.repeat(stain_prefix, n_samples, axis=0)[row, col]
new_filename = df[filename_column].loc[good_indices].as_matrix()[row]
dataset1 = ["{}{}".format(a_, b_) for a_, b_ in zip(new_prefix, new_filename)]

In [ ]:
# choose necessary stains
dataset2 = []
df_good = (df[type_columns].astype(np.object) == stain[1]).loc[good_indices, :]
row, col = np.where(df_good)
n_samples = df.shape[0]
new_prefix = np.repeat(stain_prefix, n_samples, axis=0)[row, col]
new_filename = df[filename_column].loc[good_indices].as_matrix()[row]
dataset2 = ["{}{}".format(a_, b_) for a_, b_ in zip(new_prefix, new_filename)]

In [ ]:
# choose necessary stains
dataset3 = []
df_good = (df[type_columns].astype(np.object) == stain[2]).loc[good_indices, :]
row, col = np.where(df_good)
n_samples = df.shape[0]
new_prefix = np.repeat(stain_prefix, n_samples, axis=0)[row, col]
new_filename = df[filename_column].loc[good_indices].as_matrix()[row]
dataset3 = ["{}{}".format(a_, b_) for a_, b_ in zip(new_prefix, new_filename)]

In [ ]:
def create_title(path, name_id=8):
    # get the name of the initial image
    image_name = path.split("/")[name_id] # was 8
    # print(path.split("/"))
    # create the full title 
    title = image_name[:-4]
    return title
# create_title("/Users/kkolyva/Desktop/n2/N2-results/all/C1-N2_9.csv")

In [ ]:
labels = ['DPY-23_EX'] # ['DPY-23_EX', 'WDR-5.2', 'MDH-1']
color = '#BA5536'
if labels[0] == 'MDH-1':
    color = "#693D3D"

In [ ]:
# actual plotting 
img_before = ""
if (False):
    img_before = "-before" 

dataset_to_use = dataset1
if labels[0] == 'MDH-1':
    dataset_to_use = dataset3
    
dataset = []
p_dataset = []
for j in range(0, len(dataset_to_use)):
    tmp = folder_path + "csv" + img_before + "/" + dataset_to_use[j] + ".csv"
    dataset.append(tmp)
    print(tmp)

In [ ]:
# load the parameters from file for the fitter
def load_params(file_path):
    data = np.loadtxt(file_path, delimiter = '\t', skiprows=0)
    return data

In [ ]:
# actual plotting 
for idx in range(0, len(dataset)):
    # feedback
    
    if(not os.path.exists(dataset[idx])):
        # print("doesn't exist")
        continue
    
    try:
        # print("Processing:", dataset[idx]) 
        z,I = load_data(dataset[idx], skiprows_=0)

        plt.figure(figsize=(8,5))

        title = create_title(dataset[idx], name_id=5)
        plt.title(title + " / " + labels[0])

        plt.plot(z, I, 'o', color=color)

        x_limits = [0, 120]
        y_limits = [-0.2, 1.0]

        axes = plt.gca()
        axes.set_xlim(x_limits)
        axes.set_ylim(y_limits)

        plt.xlabel('z-slice')
        plt.ylabel('intensity')

        info_text = "Total: " + str(I.shape[0])
        plt.text(x_limits[0] + (x_limits[1] - x_limits[0])*0.02, y_limits[0] + (y_limits[1] - y_limits[0])*0.04, info_text, color='black', bbox=dict(facecolor='white', alpha=1))

        # plt.legend(loc = 'upper right')      
        full_path_to_use = folder_path + "img"  + img_before + "/"+ labels[0] + "/" 
        if not os.path.exists(full_path_to_use):
            os.makedirs(full_path_to_use)
        plt.savefig(full_path_to_use + title + ".pdf")
        
        plt.show()
    except(RuntimeError, TypeError, ValueError):
        print("There was an exception but we\'ll fix it for you")